# Tensorflow Decision Forests

Link: https://www.tensorflow.org/decision_forests?hl=pt-br

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import keras
import tensorflow_decision_forests as tfdf

import tensorflow_decision_forests as tfdf

from IPython.core.magic import register_line_magic
from IPython.display import Javascript

from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

2023-03-06 21:30:09.486144: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 21:30:18.854946: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-06 21:30:18.855372: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-06 21:30:18.855411: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# Check the version of TensorFlow Decision Forests
print("Found TensorFlow Decision Forests v" + tfdf.__version__)

Found TensorFlow Decision Forests v1.2.0


In [100]:
METRICS = [ 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc')
]

tfdf.keras.get_all_models()

[tensorflow_decision_forests.keras.RandomForestModel,
 tensorflow_decision_forests.keras.GradientBoostedTreesModel,
 tensorflow_decision_forests.keras.CartModel,
 tensorflow_decision_forests.keras.DistributedGradientBoostedTreesModel]

In [101]:
df_ppi_1 = pd.read_csv('data/base_cerevisiae.csv')
df_ppi_2 = pd.read_csv('data/base_elegans.csv')
df_ppi_3 = pd.read_csv('data/base_drosophila.csv')

In [102]:
df = pd.concat([df_ppi_1, df_ppi_2, df_ppi_3], ignore_index=True)
df

,Locus,Sequence,Sequence_Length,Aromaticity,Sec_Struct_Helix,Sec_Struct_Turn,Sec_Struct_Sheet,Percent_A,Percent_C,Percent_D,...,Percent_V,Percent_W,Percent_Y,Protein_key,DegreeCentrality,EigenvectorCentrality,BetweennessCentrality,ClosenessCentrality,Clustering,is_essential
0,YPL071C,MSSRFARSNGNPNHIRKRNHSPDPIGIDNYKRKRLIIDLENLSLND...,156,0.096154,0.262821,0.435897,0.301282,0.044872,0.006410,0.128205,...,0.038462,0.032051,0.038462,4932.YPL071C,0.000986,0.000512,3.315435e-06,0.426787,0.266667,0
1,YLL050C,MSRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETST...,143,0.111888,0.293706,0.405594,0.300699,0.076923,0.006993,0.083916,...,0.083916,0.006993,0.048951,4932.YLL050C,0.053392,0.017135,2.003725e-04,0.497262,0.302355,1
2,YMR172W,MSGMGIAILCIVRTKIYRITISFDYSTLMSPFFLFLMMPTTLKDGY...,719,0.043115,0.314325,0.442281,0.243394,0.055633,0.002782,0.058414,...,0.030598,0.004172,0.018081,4932.YMR172W,0.007557,0.002314,3.144740e-06,0.438952,0.289855,0
3,YOR185C,MSAPAQNNAEVPTFKLVLVGDGGTGKTTFVKRHLTGEFEKKYIATI...,220,0.109091,0.322727,0.340909,0.336364,0.077273,0.013636,0.063636,...,0.081818,0.013636,0.040909,4932.YOR185C,0.046164,0.017683,1.040158e-04,0.491754,0.307905,0
4,YLL032C,MDNFKIYSTVITTAFLQVPHLYTTNRLWKPIEAPFLVEFLQKRISS...,825,0.100606,0.306667,0.358788,0.334545,0.042424,0.010909,0.042424,...,0.043636,0.002424,0.043636,4932.YLL032C,0.021028,0.006141,1.265410e-04,0.473128,0.236713,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30506,7227.FBpp0306211,MARLISGVRNLFHRYPFVTNSAIYGSLYVGAEYSQQFASKRWLATA...,204,0.147059,0.348039,0.269608,0.382353,0.102941,0.014706,0.029412,...,0.073529,0.029412,0.058824,7227.FBpp0306211,0.000177,0.000006,1.872049e-08,0.290879,0.000000,0
30507,7227.FBpp0306213,MVKILQAYNFARQQTYALNGDILAASLIGNNRIAISSAEQFIEIYD...,1536,0.083984,0.372396,0.333333,0.294271,0.074870,0.019531,0.052734,...,0.065104,0.005208,0.035807,7227.FBpp0306213,0.001237,0.000074,6.935191e-06,0.324301,0.648352,0
30508,7227.FBpp0306214,MSGGDYDSGDYFMRSRKQRDKPSLWDSFQDPPSKKTSGSDADWKKL...,1393,0.117014,0.384063,0.284996,0.330940,0.083274,0.022254,0.055994,...,0.071070,0.023690,0.035176,7227.FBpp0306214,0.014759,0.005067,1.719578e-04,0.421110,0.268595,0
30509,7227.FBpp0306223,MEREIAHSLAGGEERSSDVAPGQVKTFEELRLYRNLLNGLKRNNFV...,1028,0.071012,0.351167,0.381323,0.267510,0.057393,0.007782,0.052529,...,0.057393,0.003891,0.034047,7227.FBpp0306223,0.055590,0.016066,4.047679e-04,0.451477,0.227401,1


In [103]:
# Retirando identificadores

df_tmp = df.drop(['Protein_key', 'Locus', 'Sequence'], axis=1)
df_tmp

,Sequence_Length,Aromaticity,Sec_Struct_Helix,Sec_Struct_Turn,Sec_Struct_Sheet,Percent_A,Percent_C,Percent_D,Percent_E,Percent_F,...,Percent_T,Percent_V,Percent_W,Percent_Y,DegreeCentrality,EigenvectorCentrality,BetweennessCentrality,ClosenessCentrality,Clustering,is_essential
0,156,0.096154,0.262821,0.435897,0.301282,0.044872,0.006410,0.128205,0.044872,0.025641,...,0.038462,0.038462,0.032051,0.038462,0.000986,0.000512,3.315435e-06,0.426787,0.266667,0
1,143,0.111888,0.293706,0.405594,0.300699,0.076923,0.006993,0.083916,0.069930,0.055944,...,0.055944,0.083916,0.006993,0.048951,0.053392,0.017135,2.003725e-04,0.497262,0.302355,1
2,719,0.043115,0.314325,0.442281,0.243394,0.055633,0.002782,0.058414,0.047288,0.020862,...,0.083449,0.030598,0.004172,0.018081,0.007557,0.002314,3.144740e-06,0.438952,0.289855,0
3,220,0.109091,0.322727,0.340909,0.336364,0.077273,0.013636,0.063636,0.063636,0.054545,...,0.059091,0.081818,0.013636,0.040909,0.046164,0.017683,1.040158e-04,0.491754,0.307905,0
4,825,0.100606,0.306667,0.358788,0.334545,0.042424,0.010909,0.042424,0.066667,0.054545,...,0.059394,0.043636,0.002424,0.043636,0.021028,0.006141,1.265410e-04,0.473128,0.236713,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30506,204,0.147059,0.348039,0.269608,0.382353,0.102941,0.014706,0.029412,0.034314,0.058824,...,0.068627,0.073529,0.029412,0.058824,0.000177,0.000006,1.872049e-08,0.290879,0.000000,0
30507,1536,0.083984,0.372396,0.333333,0.294271,0.074870,0.019531,0.052734,0.065104,0.042969,...,0.046875,0.065104,0.005208,0.035807,0.001237,0.000074,6.935191e-06,0.324301,0.648352,0
30508,1393,0.117014,0.384063,0.284996,0.330940,0.083274,0.022254,0.055994,0.048816,0.058148,...,0.045944,0.071070,0.023690,0.035176,0.014759,0.005067,1.719578e-04,0.421110,0.268595,0
30509,1028,0.071012,0.351167,0.381323,0.267510,0.057393,0.007782,0.052529,0.081712,0.033074,...,0.053502,0.057393,0.003891,0.034047,0.055590,0.016066,4.047679e-04,0.451477,0.227401,1


In [104]:
# Separação em conjuntos de treino e teste
X = df.drop(['Protein_key','is_essential', 'Locus', 'Sequence'], axis=1)
#X = df_select
y = df[['is_essential']]

# Name of the label column.
label = "is_essential"

test_size = 0.2
seed = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=test_size, 
                                                    random_state=seed, 
                                                    stratify=y)


In [105]:
## Combine


#X_train = train_ds_pd.drop(['is_essential'], axis=1) 

#y_train = train_ds_pd.drop(X_train.columns, axis=1)

#sample = SMOTEENN(random_state=seed)
#sample = RandomUnderSampler(random_state=seed)
sample = SMOTE(random_state=seed)

X_train_sample, y_train_sample = sample.fit_resample(X_train, y_train)


In [106]:
train_ds_pd = pd.concat([X_train_sample, y_train_sample], axis=1)
test_ds_pd = pd.concat([X_test, y_test], axis=1)

print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

46148 examples in training, 6103 examples for testing.


In [107]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task=tfdf.keras.Task.CLASSIFICATION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label, task=tfdf.keras.Task.CLASSIFICATION)

### Treinamento

In [120]:
# Specify the model.
model_1 = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.CLASSIFICATION, 
                                       num_trees = 300,
                                       max_depth = 7
                                       )

# Optionally, add evaluation metrics.
model_1.compile( 
    metrics=[METRICS])

# Train the model.

model_1.fit(x=train_ds)


model_1.save("project/model")

Use /tmp/tmp2cz64kma as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.921368. Found 46148 examples.
Training model...
Model trained in 0:00:04.659440
Compiling model...


[INFO 2023-03-06T22:15:05.250400606-03:00 kernel.cc:1214] Loading model from path /tmp/tmp2cz64kma/model/ with prefix 2e81ec7c02c4404e
[INFO 2023-03-06T22:15:05.315616424-03:00 decision_forest.cc:661] Model loaded with 300 root(s), 32538 node(s), and 30 input feature(s).
[INFO 2023-03-06T22:15:05.31564406-03:00 abstract_model.cc:1311] Engine "RandomForestOptPred" built
[INFO 2023-03-06T22:15:05.315658617-03:00 kernel.cc:1046] Use fast generic engine


Model compiled.


INFO:tensorflow:Assets written to: project/model/assets


INFO:tensorflow:Assets written to: project/model/assets


In [121]:
model_1.summary()

Model: "random_forest_model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (30):
	Aromaticity
	BetweennessCentrality
	ClosenessCentrality
	Clustering
	DegreeCentrality
	EigenvectorCentrality
	Percent_A
	Percent_C
	Percent_D
	Percent_E
	Percent_F
	Percent_G
	Percent_H
	Percent_I
	Percent_K
	Percent_L
	Percent_M
	Percent_N
	Percent_P
	Percent_Q
	Percent_R
	Percent_S
	Percent_T
	Percent_V
	Percent_W
	Percent_Y
	Sec_Struct_Helix
	Sec_Struct_Sheet
	Sec_Struct_Turn
	Sequence_Length

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.   "ClosenessCentrality"  0.252845 ################
    2. "EigenvectorCentrality"  0.214977 ##########
    3.      "DegreeCentrality"  0.213577 ##########
    4. "Betw

### Evaluation

In [122]:
results = model_1.evaluate(test_ds, return_dict=True, verbose=0)
print("model_1 Evaluation: \n")
for name, value in results.items():
    print(f"{name}: {value:.4f}")

model_1 Evaluation: 

loss: 0.0000
accuracy: 0.6862
precision: 0.1218
recall: 0.7625
auc: 0.7966


In [123]:
results

{'loss': 0.0,
 'accuracy': 0.6861652731895447,
 'precision': 0.12181122601032257,
 'recall': 0.7624750733375549,
 'auc': 0.7966101169586182}

In [124]:
#tfdf.model_plotter.plot_model(model_1, tree_idx=0, max_depth=3)


with open("plot.html", "w") as f: 
    f.write(tfdf.model_plotter.plot_model(model_1, tree_idx=0, max_depth=5))


In [126]:
# Avaliação do Modelo

from sklearn.metrics import classification_report

test_predictions = np.round(model_1.predict(test_ds))

test_values = np.array(test_ds_pd['is_essential'])

# Comparing the predictions to actual forest cover types for the test rows
# test is the data right after splitting into train, test and val 
#(shuffle was false in dataset so the order will match)

print(classification_report(test_values,test_predictions))


7/7 [==============================] - 0s 14ms/step
              precision    recall  f1-score   support

           0       0.98      0.68      0.80      5769
           1       0.12      0.76      0.21       334

    accuracy                           0.69      6103
   macro avg       0.55      0.72      0.51      6103
weighted avg       0.93      0.69      0.77      6103



In [74]:
# Export the meta-data to tensorboard.
model_1.make_inspector().export_to_tensorboard("/tmp/tensorboard_logs")

In [75]:
# docs_infra: no_execute
# Start a tensorboard instance.

%load_ext tensorboard

%tensorboard --logdir "/tmp/tensorboard_logs"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 6679), started 0:18:37 ago. (Use '!kill 6679' to kill it.)

# Neural Networks

In [155]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier

In [164]:
# Function to create model
def create_model(neurons, dim):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_dim=dim))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])
    
    return model

In [165]:
# build the model
model = create_model(100, X_train_sample.shape[1])

model.fit(X_train_sample, y_train_sample, epochs=50)

Epoch 1/50
1443/1443 [==============================] - 2s 1ms/step - loss: 1.0787 - accuracy: 0.5412
Epoch 2/50
1443/1443 [==============================] - 2s 1ms/step - loss: 0.7690 - accuracy: 0.5851
Epoch 3/50
1443/1443 [==============================] - 2s 1ms/step - loss: 0.7208 - accuracy: 0.6182
Epoch 4/50
1443/1443 [==============================] - 2s 1ms/step - loss: 0.7022 - accuracy: 0.6394
Epoch 5/50
1443/1443 [==============================] - 2s 1ms/step - loss: 0.6954 - accuracy: 0.6516
Epoch 6/50
1443/1443 [==============================] - 2s 1ms/step - loss: 0.6951 - accuracy: 0.6552
Epoch 7/50
1443/1443 [==============================] - 2s 1ms/step - loss: 0.6843 - accuracy: 0.6639
Epoch 8/50
1443/1443 [==============================] - 2s 1ms/step - loss: 0.6725 - accuracy: 0.6709
Epoch 9/50
1443/1443 [==============================] - 2s 1ms/step - loss: 0.6666 - accuracy: 0.6761
Epoch 10/50
1443/1443 [==============================] - 2s 1ms/step - loss: 0.681

In [166]:
pred_train= model.predict(X_train_sample)
scores = model.evaluate(X_train_sample, y_train_sample, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   
 
pred_test= model.predict(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))    

1443/1443 [==============================] - 1s 739us/step
Accuracy on training data: 0.5935685038566589% 
 Error on training data: 0.40643149614334106
191/191 [==============================] - 0s 750us/step
Accuracy on test data: 0.8984106183052063% 
 Error on test data: 0.1015893816947937


In [167]:
from sklearn.metrics import classification_report

pred_test = np.round(pred_test)

test_values = np.array(test_ds_pd['is_essential'])

# Comparing the predictions to actual forest cover types for the test rows
# test is the data right after splitting into train, test and val 
#(shuffle was false in dataset so the order will match)

print(classification_report(test_values,pred_test))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95      5769
           1       0.15      0.19      0.17       334

    accuracy                           0.90      6103
   macro avg       0.55      0.57      0.56      6103
weighted avg       0.91      0.90      0.90      6103

